In [1]:
from mlaut.analyze_results import AnalyseResults
from mlaut.data import Data
from mlaut.data.data import Data
import pickle
import numpy as np
from sklearn.metrics import mean_squared_error
from mlaut.experiments import Orchestrator
from mlaut.estimators.estimators import instantiate_default_estimators
import os

In [2]:
data = Data()
input_io = data.open_hdf5('data/delgado.h5', mode='r')
out_io = data.open_hdf5('data/delgado-classification-deep.h5', mode='a')
analyze = AnalyseResults(hdf5_output_io=out_io, 
                        hdf5_input_io=input_io, 
                        input_h5_original_datasets_group='openml/', 
                        output_h5_predictions_group='experiments/predictions/')

In [3]:
dts = out_io.list_datasets('experiments/predictions')

In [6]:
for d in dts:
    estimaotrs_list = out_io.list_datasets(f'experiments/predictions/{d}')
    for e in estimaotrs_list:
        p = out_io.load_dataset_h5(f'experiments/predictions/{d}/{e}')
        if p[0].ndim > 1:
            new_p = p[0].argmax(axis=1)
            out_io.save_prediction_to_db(predictions=new_p, dataset_name=d, strategy_name=e)
            print(f'Overriding: {d}/{e}')
                



Overriding: abalone/NN-12-layer_wide_with_dropout
Overriding: abalone/NN-4-layer_thin_dropout
Overriding: abalone/NN-4-layer_wide_no_dropout
Overriding: abalone/NN-4-layer_wide_with_dropout
Overriding: acute_inflammation/NN-12-layer_wide_with_dropout
Overriding: acute_inflammation/NN-4-layer_thin_dropout
Overriding: acute_inflammation/NN-4-layer_wide_no_dropout
Overriding: acute_inflammation/NN-4-layer_wide_with_dropout
Overriding: acute_nephritis/NN-12-layer_wide_with_dropout
Overriding: acute_nephritis/NN-4-layer_thin_dropout
Overriding: acute_nephritis/NN-4-layer_wide_no_dropout
Overriding: acute_nephritis/NN-4-layer_wide_with_dropout
Overriding: adult/NN-12-layer_wide_with_dropout
Overriding: adult/NN-4-layer_thin_dropout
Overriding: adult/NN-4-layer_wide_no_dropout
Overriding: adult/NN-4-layer_wide_with_dropout
Overriding: annealing/NN-12-layer_wide_with_dropout
Overriding: annealing/NN-4-layer_thin_dropout
Overriding: annealing/NN-4-layer_wide_no_dropout
Overriding: annealing/NN-

In [ ]:
p = out_io.load_dataset_h5(f'experiments/predictions/abalone/SVC')

In [7]:
p = out_io.load_dataset_h5(f'experiments/predictions/abalone/NN-12-layer_wide_with_dropout')

In [8]:
p[0]

array([0, 0, 0, ..., 0, 0, 0])

In [9]:
p[0].ndim

1

In [ ]:
len(p[0].argmax(axis=1))